In [4]:
import json
import numpy as np
import pandas as pd
import os
import glob
import shutil
from tqdm import tqdm
#import japanize_matplotlib
import matplotlib.pyplot as plt
#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'
#from sciplotlib import style as spstyle
import seaborn as sns
from sklearn.model_selection import train_test_split,StratifiedGroupKFold

def search_object(sentence,organ):
    ##構造化文章単位を入力として、特定臓器の情報が入っているかを調べる。
    ##入力 sentence
    ##出力  IO+AEのtoken,IO token,AE token,IOのcertainty_score,IOのtype
    img_obs = sentence['clinical_object']
    mdfs = sentence['modifiers']
    ae_tokens = []

    for i in range(len(mdfs)):
        mdf = mdfs[i]
        if mdf['type'] == 'Anatomical_entity':
            ae_tokens += mdf['tokens']
    
    keywords = ae_tokens+img_obs['tokens']
    #print(keywords)
    if organ in keywords:
        keywords = ''.join(ae_tokens+img_obs['tokens'])
        io_tokens = ''.join(img_obs['tokens'])
        ae_tokens = ''.join(ae_tokens)
        return keywords,io_tokens,ae_tokens,img_obs['certainty_score'],img_obs['type']
    else:
        return None,None,None,None,None
def search_clinicalfinding(modifiers):
    ##clinical_descriptor clinical_discriptorが文中に含まれているかを判別
    ##入力 modifiers
    ##出力  含まれるかどうかbool,clinicalと判断されたwords(なければ0を入れる。)
    for i in range(len(modifiers)):
        modifier = modifiers[i]
        if modifier['type'] == 'Clinical_finding':
            return modifier['certainty_score'],''.join(modifier['tokens'])
        else:continue
    return None,'not_found'
def to_multi_label(df,abnormal_list):
    for abnormal_label in abnormal_list:
        df[abnormal_label] = df['label'].apply(lambda x: abnormal_label in x).astype(int)
    df['nofinding'] = df[abnormal_list].sum(axis=1) == 0
    return df

In [5]:
###全体のラベル作成
###observationは絶対に文中にあるという前提。
###

df = pd.read_pickle('../server_data/all_data_20211001-20220605.zip')

organs = ['心','肝','胆','膵','脾','副腎','腎']
os.makedirs('../output',exist_ok=True)
counter = 0
jmid_names = df['jmid_name']
json_lists = df['FINDING_JSON']
#def detect_change(modifiers):
change_word_list = []
clinical_word_list = []
data = []
counter = 0
for jmid_name,json_load in zip(jmid_names,json_lists):
    for organ in organs:
        #print(organ)
        for i in range(len(json_load)):
            all_keyword,io_keyword,ae_keyword,certainty,img_type = search_object(json_load[i],organ)
            clinical_scale,clinical_word = search_clinicalfinding(json_load[i]['modifiers'])

            
            if all_keyword is not None:
                counter+=1
                data.append([jmid_name,organ,all_keyword,io_keyword,ae_keyword,certainty,img_type,clinical_word,clinical_scale])
            else:
                continue

data = pd.DataFrame(data,columns=['jmid_name','organ','all_tokens','io_tokens','ae_tokens','obs_certainty','obs_name','clin_findings','clin_certainty'])
data['jmid_name'] = data['jmid_name'].apply(lambda x:x.split('/')[-1])

In [6]:
data_liver = data[(data['obs_name'] == 'Imaging_observation') & (data['organ'] == '肝')].reset_index(drop=True)

In [7]:
print([i for i in data_liver[data_liver['obs_certainty'] == 0]['io_tokens'].value_counts().index])

['SOL', '肝SOL', '異常', '所見', '病変', '腫瘤影', '肝腫瘤', '腫瘤', '脂肪肝', '腫瘤性病変', '肝内胆管拡張', '血栓', '肝脾腫', 'washout', '狭窄', '拡張', '腫瘍栓', '早期濃染', '像', '異常所見', '腫瘤形成', '染まり抜け', '占拠性病変', '有意な異常', '結節', 'wash_out', 'リンパ節腫大', '腫瘍', '洗い出し', '造影効果', '閉塞機転', '肝充実性腫瘤', 'LDA', '新出病変', '肝内SOL', '腫大', '肝腫大', '粗大病変', '血管腫', '液体貯留', '造影不良', '多血化', '肝病変', '腹水', '後期染まり抜け', '早期濃染像', '肝内結石', '早期濃染域', '拡張像', '造影不良域', '液貯留', '膿瘍', '早期濃染病変', '破格', '閉塞', '肝損傷', '内部壊死', '被包化', '粗大腫瘤', '濃染', '結石', '低吸収域', '拡大', '膜様構造', '顕在化', '副路発達', '開存', '脂肪濃度', '目立たず', '肝変形', '有意な肥厚', '古典的', 'anomaly', '点状早期濃染', '壁肥厚', '小LDA', '軟部影', '狭窄胆管拡張', '高吸収域', '肝膿瘍形成', '早期相濃染', '結節影', '異常濃染域', '浮腫', '凹凸不整', '肝内腫瘤性病変', '造影遅延', '前立腺腫大', 'encasement', '血流異常', '脈管侵襲', '総胆管拡張', '肝内病変', '肝内腫瘤', 'viableHCC', '肝嚢胞', 'biloma形成', '異常造影効果', '異常造影結節', '早期濃染不明瞭', '異常胆管拡張像', '造影ムラ', '結節状造影効果', 'washout像', '造影不領域', '早期造影', '腫瘍濃染像', '胆管拡張', 'extravasation', '膿瘍形成', '狭小化', '浸潤像', 'RFA後病変', '門脈腫瘍栓', '総肝管拡張', '肝占拠性病変', '異常濃度域', '小嚢胞', 'ヘルニア状突出', '低濃度域', '異常壁肥厚像', 

In [8]:
normal_obs_finding_list = ['SOL', '肝SOL', '異常', '所見', '病変', '腫瘤影', '肝腫瘤', '腫瘤', '脂肪肝', '肝内胆管拡張', '腫瘤性病変', 'washout', 
                        '早期濃染', '像', '異常所見', '腫瘤形成', '染まり抜け', '占拠性病変', '有意な異常', '結節', 'wash_out', '腫瘍', 
                        '洗い出し', '造影効果', '閉塞機転', '肝充実性腫瘤', 'LDA', '新出病変', '肝内SOL', '肝腫大', '腫大', '粗大病変', '造影不良', '血管腫', '液体貯留', 
                        '肝病変', '拡張像', '早期濃染域', '早期濃染像', '後期染まり抜け', '肝内結石', '造影不良域', '液貯留', '早期濃染病変', '膿瘍', '破格', '結石', '粗大腫瘤', 
                        '濃染', '肝損傷', '閉塞', '被包化', '内部壊死', '拡大', '低吸収域', '肝変形', '膜様構造', '有意な肥厚', '顕在化', '目立たず', '開存', '脂肪濃度', '副路発達', '軟部影', 
                        '小LDA', 'anomaly', '点状早期濃染', '高吸収域', '古典的', '壁肥厚', '浮腫', '凹凸不整', '結節影', '異常濃染域', '早期相濃染', '肝内腫瘤性病変', 
                        '造影遅延', '肝膿瘍形成', '血流異常', '肝内病変', '腫瘍濃染像', 'biloma形成', '狭小化', '膿瘍形成', 
                        '異常造影結節', 'viableHCC', 'RFA後病変', '浸潤像', '結節状造影効果', '肝内腫瘤', '早期濃染不明瞭', '早期造影', '造影不領域', '異常造影効果', 'washout像', 
                        '肝嚢胞', '肝占拠性病変', '早期濃染まり', '小嚢胞', '異常壁肥厚像', '異常濃度域', '肝LDA', '低濃度域','早期濃染の所見', 
                        '低吸収結節', '骨病変肝脾腫', '小結節', '事項', '肝脾腫大', '肝内胆管拡張像', '膿瘍腔', '斑状早期濃染', '造影効果低下域', 'ただちに異常', 
                        '肝内LDA', '嚢胞構造', 'うっ血肝', '石灰化', '肝内結石の新出', '異常ない異常', 'mass', '浮腫性変化', '肝実質低吸収域', 
                        '嚢胞状病変', '脂肪肝の新出', '嚢胞性病変']

In [9]:
print([i for i in data_liver[data_liver['obs_certainty'] != 0]['io_tokens'].value_counts().index])

['肝嚢胞', '脂肪肝', '嚢胞', '拡張', '小嚢胞', '早期濃染', '開存', '肝内胆管拡張', '腫瘤', 'LDA', '血管腫', '早期濃染域', '造影効果', '結節', '軟部影', 'リンパ節', '液貯留', 'SOL', '低吸収域', 'リンパ節腫大', '狭小化', '肝のう胞', '腹水', '液体貯留', '肝血管腫', '早期濃染結節', '病変', '石灰化', '微小嚢胞', '腫大', '早期濃染像', '狭窄', 'pneumobilia', '造影不良域', '低吸収', '肝腫大', '濃染', '小結節', '胆管拡張', '低濃度域', '壁肥厚', '乏血性結節', '小リンパ節', '血腫', '変形', '早期造影域', '肝石灰化', 'RFA後変化', '小肝嚢胞', '血流', '肝腫瘤', '肝変形', '萎縮', 'washout', '肝縁鈍化', '腫大リンパ節', '低吸収結節', '肝形態', 'wash_out', '腫瘤影', '微小肝嚢胞', '小石灰化', '腎嚢胞', '閉塞', '低濃度', '結節状早期濃染', '拡張像', '分岐しています', '高吸収域', '発達', '造影不良', '胆石', '高吸収', '嚢胞性病変', '肝SOL', '脾腫', '結節状の早期濃染', '腫瘍栓', '像', '腫瘍', '腫瘤性病変', '軟部濃度', '結節状', '肝萎縮', 'AP_shunt', '変形萎縮', '遠肝性側副路発達', 'PV_shunt', '濃染域', '顕在化', '解離', '凹凸不整', '蛇行', '結節影', '浮腫', '被包化液貯留', '肝硬変', 'air', '造影域', '小血管腫', '軟部組織陰影', '濃染結節', '腹水貯留', '肝内胆管拡張像', '脂肪化', '鈍化', '軟部濃度域', '圧排', '肝内石灰化', '脂肪織混濁', '小早期濃染像', '軟部組織', '肝内外胆管拡張', 'PVシャント', '早期増強効果', 'P-Vシャント', '軟部陰影', '造影結節', '総胆管拡張', '濃度上昇', '脂肪沈着', '肝脾腫', '造影効果は不均一', '肝内結石', '肝病変', '

**abnormal_obs_finding_list**→異常なラベルとして使用するもの  
**abnormal_obs_finding_id**→異常の大まかなカテゴリ付け。  
**abnormal_obs_finding_renamed**→大まかなカテゴリの名前

In [ ]:
abnormal_obs_finding_list = ['肝嚢胞', '脂肪肝', '嚢胞', '拡張', '小嚢胞', '早期濃染', '開存', '肝内胆管拡張', '腫瘤', 'LDA', '血管腫', '早期濃染域', '造影効果', '結節', '軟部影', 'リンパ節', '液貯留', 'SOL', '低吸収域', '狭小化', 'リンパ節腫大', '肝のう胞', '腹水', '液体貯留', '早期濃染結節', '肝血管腫', '病変', '石灰化', '微小嚢胞', '腫大', '早期濃染像', '狭窄', 'pneumobilia', '造影不良域', '低吸収', '肝腫大', '濃染', '小結節', '胆管拡張', '壁肥厚', '低濃度域', '乏血性結節', '小リンパ節', '血腫', '変形', '早期造影域', '肝石灰化', 'RFA後変化', '小肝嚢胞', '血流', '肝腫瘤', '肝変形', '萎縮', 'washout', '肝縁鈍化', '腫大リンパ節', '肝形態', '低吸収結節', 'wash_out', '腫瘤影', '小石灰化', '微小肝嚢胞', '腎嚢胞', '閉塞', '低濃度', '結節状早期濃染', '分岐しています', '拡張像', '高吸収域', '発達', '高吸収', '胆石', '造影不良', '嚢胞性病変', '腫瘍栓', '結節状の早期濃染', '像', '脾腫', '肝SOL', '腫瘍', '結節状', 'AP_shunt', '腫瘤性病変', '軟部濃度', '肝萎縮', '変形萎縮', '遠肝性側副路発達', 'PV_shunt', '濃染域', '解離', '顕在化', '凹凸不整', '蛇行', '結節影', '浮腫', '被包化液貯留', '肝硬変', 'air', '造影域', '小血管腫', '濃染結節', '軟部組織陰影', '腹水貯留', '肝内胆管拡張像', '脂肪織混濁', '小早期濃染像', '肝内石灰化', '軟部濃度域', '圧排', '鈍化', '脂肪化', '早期増強効果', '軟部組織', '肝内外胆管拡張', '軟部陰影', 'PVシャント', 'P-Vシャント', '肝脾腫', '造影結節', '造影効果は不均一', '脂肪沈着', '濃度上昇', '総胆管拡張', 'グラフト', '肝病変', '染まり抜け', 'free_air', '肝内結石', '動脈瘤', '被包化', '早期造影効果', '腫瘤形成', 'normal_size_and_shape', '増強効果', '造影不領域', '求肝性', '血栓', 'LDAs', '造影効果まだら', '造影効果乏しく', '肝内胆管拡張軽減', 'ステントの開存性', '小突出', '遠肝性側副血行路', 'ドレーンチューブ', '血流異常', '分岐', '血性腹水', '浮腫性変化', '肝内胆管気腫', '結石', '遠肝性側副血行路発達', '肝内病変', '突出像', '増強不良域', '小LDA', '陥凹', '肝結節', '肝膿瘍', '微小結節', 'HCCs', '分岐するパターン', '小早期濃染', 'カテーテル留置', '後期染まり抜け', '所見', '副路発達', '遠肝性', '異常造影効果', 'Air', '低吸収化', '濃度', '造影ムラ', '乏血性病変', '焼灼域', '造影', '胸水（-）肝', 'リピオドール集積', '嚢胞性陰影', '小結節状領域', '肝外胆管拡張', '嚢胞変性', '肝吸収値', '遠肝性側', '早期濃染し後期洗い出し', '口径不整', '発達像', '結節状の濃染', '微小石灰化', '濃染像', '造影パターン', '造影効果低下', '造影効果がまだら', '脂肪濃度', 'cavity', '脂肪性結節', '遠肝性側副路', '造影効果は低下', '貯留', '軟部濃度組織', '肥厚部', 'コイル', '集積', '肥厚', '結節状構造', '結腸憩室', '被包化液体貯留', '突出', '腹水増加', '早期濃染領域', '造影効果良好', '線状の造影欠損領域', '壁肥厚像', 'cavernous_transformation', '静脈拡張', '辺縁鈍化', '乏血性', '造影遅延', '遠肝性側副血行路形成', '狭細化', '脂肪腫', '高濃度', '破格（-）胆石', '液体', '大部分脂肪濃度', '脈管開存', '胆道気腫', '領域', 'lipiodol沈着', 'spared_lesion', '壊死状', '肝内胆管拡張±胆', '広狭不整', '萎縮変形', '静脈内腫瘍栓', 'densityの低下', '吸収値高く', '遊離ガス', '小濃染域', '乏血性な領域', '後期相染まり抜け', '門脈瘤', '早期', 'lipiodol沈着部', '造影効果の残る領域', 'ドレーン', '浮腫状壁肥厚', '鋭', '静脈早期還流', '造影剤の流入', '軟部組織濃度', '血管筋脂肪腫', '遠肝性側副血行の発達', 'enhanced_nodule', '門脈開存', 'fill-in', '胆管チューブ', '胆嚢腫大', '高濃度域', '凹凸', '起始', '壊死変性', 'ステント', '濃染パターン', '胆嚢床', '地図状脂肪肝', '軽度胆管拡張', '胸水', '脂肪濃度結節', '造影増強効果', '液貯留腔', '脂肪織の毛羽立ち', '胆管tube', 'walled-off_necrosis', '腹膜肥厚', '共通幹', 'リンパ節腫大（-）肝', '低信号域', 'CCC', '早期濃染腫瘤', '早期造影結節', '軽度肝内胆管拡張', '肝嚢胞不変', 'A-P_shunt', '細化', '金属マーカー', '低濃度病変', '主膵管拡張', 'たわみ', '腫瘍血管の発達', '外側域', '治療瘢痕', '末梢胆管拡張', '嚢胞状の腔', '吸収値', '脂肪濃度の結節', 'density', 'Lpd集積', 'ENBD_tube', '膿瘍', '肝低濃度結節', '大腸憩室', '肝形態変化', '肝内胆管狭窄', 'リピオドール貯留', '造影腫瘤', '病変内', '高吸収な液貯留', '門脈ガス', '嚢状瘤', '構造', '早期還流', '造影欠損', '肝下面膿瘍', '濃染域の広がり', 'washoutする領域', '早期濃染-wash_out', '斑状早期濃染', '早期分岐', '鈍化変形', '求肝性側副路発達', '肝内動脈血流', 'AP-shunt', '造影早期', 'APシャント', '造影効果はまだら', '脂肪肝の像', '染まり方', '肝門部胆管病変', '脂肪変性像', '辺縁', '胃静脈瘤肝', '口径', '微小結節状の早期濃染', 'リピオドール沈着', '増強域', '脂肪肝高度', '脂肪濃度混濁', '血管腫様', '造影後早期', '仮性動脈瘤', '胆管気腫像', '乏血性の結節', 'pseudocirrhosis', '像（-）肝', '狭窄部', '結節様', '肝部分', '低濃度結節', '胆管拡張（-）地図状脂肪肝', '遠肝性側副路増生', '地図状濃染', '側副路発達', '小病変', '造影効果不良', 'cTACE後', '緊満感', 'くびれ', '造影効果不良域', '造影病変', '焼灼域内', '索状影', '肝囊胞', '非脂肪沈着', '嚢胞ないし血管腫', 'encasement', '菲薄化', '胆管腫瘍栓', 'wash_out域', '沿う淡い染まり', 'スパズム様変化', '気腫', 'ring_enhancement', '壁', '淡く早期濃染後', '肥大', '早期濃染／washout', '梗塞像', '共通管', '脂肪結節', '拡張軽減', '狭窄像', '小石灰化像', '結節状早期濃染域', '早期濃染濃染域', 'たり', '内部濃染増強', '腫瘤影（-）胆管拡張', '充実部分', '瘤状拡張', '腹壁肥厚', 'グリソン浮腫', '結節状早期濃染病変', '液体貯留像', 'ステント留置', '脂肪肝（-）', '濃度は不均一', '照射域', 'PTCD_tube', 'Lip貯留', '肝LDA', '不染域', '肝内コントラスト不良', 'dull', '血行動態', 'びPneumobilia', '後期洗い出し', '肝辺縁鈍化', '早期描出', '肝損傷', '肝門部胆管壁肥厚', '肉芽腫', '瘤', '細径化', '洗い出し', '嚢胞様腫瘤', '液', 'cavity残存', 'ガス貯留', '濃染する領域', '代償性発達', 'Wash_out', '造影遅延域', '隔壁構造', 'RFA変化', 'lipiodol', '描出不良', '軽度肝内胆管拡張像', '早期濃染する領域', 'FDG集積', '瘤状の拡張', '肝門部リンパ節腫大', '結節状病変', '総肝', '早期濃染腫瘤影', '異常集積', '小結節状の早期濃染域', '濃染し洗い出し', 'つながるair', '肝硬変の像', 'encase', '早期濃染部位', '濃染部', 'RFA後瘢痕', '嚢胞状拡張', '濃度低下', '肝グリソン浮腫', '陰影', '造影効果が低下', '肝LDAs', '枝状air', '早期造影不領域', '脂肪肝破壊前', '造影効果は均一', '病変小さく', '肝早期濃染像', '胆管気腫', '早期相にて濃染', '胆管ステント', '外側区域低吸収域', 'うっ血肝', 'pneumobiliawp', '早期濃染病変', '早期造影病変', '巨大肝嚢胞', '造影剤', '高吸収結節', 'P-V_shunt', '結節構造', '脂肪', '吸収域', '膿瘍腔', '小SOL', '巨大嚢胞', '巨大腫瘤', '粗大病変', 'Lipiodol集積', '輝度上昇', '血管腫病変', '胸水（-）肝血管腫', '形態変化', '小増強', '肝内低濃度域', '結節性過形成', 'アーチファクト', '小低吸収域', '肝縁の鈍化', 'リピオドール', '低吸収病変', 'cavemous_transformation', '小結節影', '内部densityは低下', 'Lipiodol貯留', '結節性病変', '濃度も不均一', 'peripheral_globular_enhancement',  '微小低吸収域', '結節状集積', '浮腫像', '嚢胞状の構造', 'SOL様', '索状LDA', '沈着','低吸収な領域', '浮腫状肥厚', '圧迫', '高集積', '脂肪濃度腫瘤', '濃度が上昇', '小葉肝隔壁の肥厚', '空洞', '変性', '軽度肝', '肝内低吸収域', '前方への突出', '出血', '低吸収なSOL', '肝機能低下', '嚢補', 'shunt', '陥凹（scalloping）', '心拡大（-）肝嚢胞', '低吸結節', '異物', '小サイズのLDA', '総胆管結石ERCP後肝内胆管拡張', '高吸収菜帯状', '肝内低濃度結節', '主病変', 'AML', '点状石灰化像', '脂肪織の濃度上昇', '低吸収の陰影', '上昇', '肝periportal_collar', '取り込み低下結節', 'pseudolipoma', '軟部組織の増大', '低濃度SOL', 'リング状濃染域', '肝低吸収域', 'ガス像', '内部吸収値が上昇', '造影剤貯留', '濃度改善', '脂肪肝改善', '脂肪性腫瘤', '肝点状石灰化', 'peumobilia', '低吸収SOL', 'ENBD', '内外瘻後脂肪肝', '異常濃度域', '瘢痕像', '肝門部液貯留', '微小石灰', 'debris', '胆管内気腫', '石灰化病変', '肝実質高吸収', '表面凹凸', '肝内SOL', '脂肪成分', '肝表不整', '対称肝', '索状の低吸収域', '濃度は均一', 'ガス', '吸収値上昇', '粗大石灰化', '肝内胆管拡張残存']

abnormal_obs_finding_list = ['肝嚢胞', '脂肪肝', '嚢胞', '拡張', '小嚢胞', '早期濃染', '肝内胆管拡張', '腫瘤', 'LDA', '血管腫', '早期濃染域', '造影効果', '結節', '軟部影', '液貯留', 'SOL', '低吸収域',
                        '肝のう胞', '液体貯留', '肝血管腫', '早期濃染結節', '病変', '石灰化', '微小嚢胞', '腫大', '早期濃染像', '狭窄', 'pneumobilia', '造影不良域', '低吸収', '肝腫大', '濃染', '小結節',
                        '胆管拡張', '低濃度域', '乏血性結節','変形', '早期造影域', '肝石灰化', 'RFA後変化', '小肝嚢胞',  '肝腫瘤', '肝変形', '萎縮', 'washout', '肝縁鈍化', '低吸収結節', '肝形態', 'wash_out',
                        '腫瘤影', '微小肝嚢胞', '小石灰化', '低濃度', '結節状早期濃染', '拡張像', '高吸収域', '造影不良', '高吸収', '嚢胞性病変', '肝SOL', '結節状の早期濃染', '腫瘍', '腫瘤性病変', '軟部濃度',
                        '結節状', '肝萎縮']
abnormal_obs_finding_list = ['肝嚢胞', '脂肪肝', '嚢胞', '拡張', '小嚢胞', '早期濃染', '肝内胆管拡張', '腫瘤', 'LDA', '血管腫', '早期濃染域', '結節',
                            '液貯留', 'SOL', '低吸収域', '狭小化', '肝のう胞', '早期濃染結節', '肝血管腫', '病変', '石灰化', '微小嚢胞', 
                            '腫大', '早期濃染像', '狭窄', 'pneumobilia', '造影不良域', '低吸収', '肝腫大', '濃染', '小結節', '低濃度域', '乏血性結節' 
                            '血腫', '変形', '早期造影域', '肝石灰化', 'RFA後変化', '小肝嚢胞', '血流', '肝腫瘤', '肝変形', '萎縮', 'washout', '肝縁鈍化', '低吸収結節', 
                            'wash_out', '腫瘤影', '小石灰化', '微小肝嚢胞','低濃度', '結節状早期濃染', '高吸収域',  '高吸収', '造影不良', 
                            '嚢胞性病変', '結節状の早期濃染', '像', '肝SOL', '腫瘍', '結節状', 'AP_shunt', '腫瘤性病変']
###normal findingsの所見idは0として、abnobal findingsの所見idをカテゴリごとに割り振る(i.e. 嚢胞→１、石灰化→2,脂肪肝→3などなど)
### マルチラベルにする。
##正常として使うラベルを指定する。
#normal_obs_finding_list = ['SOL', '異常', '所見', '病変', '腫瘤影', '腫瘤', '腫瘤性病変','異常所見','占拠性病変', '有意な異常','新出病変','washout']
##異常として使うラベルを指定する。
#abnormal_obs_finding_list = ['嚢胞', '拡張', '小嚢胞', '腫瘤', 'LDA', '血管腫', '脂肪肝', '石灰化','肝嚢胞', '早期濃染像', 'pneumobilia']
##疾患のグループにまとめる。
abnormal_obs_finding_id = [0,1,0,2,0,8,2,3,3,4,8,8,3,8,8,3,3,0,8,4,3,8,6,0,8,3,8,7,3,3,5,8,3,2,3,3,5,3,6,8,0,3,5,5,3,5,3,5,3,3,0,6,3,3,2,3,3,3,0,3,3,3,3,8,3,5]
abnormal_obs_finding_id = [0,1,0,2,0,4,2,3,3,4,4,4,3,4,4,3,3,0,4,4,3,4,4,0,4,3,4,4,3,3,4,4,3,2,3,3,4,3,4,4,0,3,4,4,3,4,3,4,3,3,0,4,3,3,2,3,3,3,0,3,3,3,3,4,3,4]

#abnormal_obs_finding_id = [0,2,0,2,0,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,0,2,2,2,2,2,2,2]
#abnormal_obs_finding_id = [i !=0 for i in abnormal_obs_finding_id]
abnormal_obs_finding_id = [0,1,0,2,0,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,0,2,2,2,2,2,2,2]
abnormal_obs_finding_id = [0,1,0,3,0,2,3,2,2,2,2,2,2,2,2,2,0,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,0,2,2,2,2,2,2,2]
##グループのまとまりの名前をつける。まとまりごとに文字の包含関係がないようにする。→嚢胞と肝嚢胞というグループを作らない。
abnormal_obs_finding_list_renamed = ['嚢胞', '脂肪肝', '胆管拡張', 'SOL',  '血管腫', '変形', '石灰化','pneumobilia','other_abnormality']
abnormal_obs_finding_list_renamed = ['嚢胞', '脂肪肝', 'other_abnormality']
#abnormal_obs_finding_list_renamed = ['嚢胞', 'other_abnormality']
abnormal_obs_finding_list_renamed = ['嚢胞', '脂肪肝', 'other_abnormality','拡張']

**セグメンテーションが使えるもの　かつ　異常(正常)ラベルが定義されているもの**を学習に使用  
データは検査ごとにまとめる。


In [ ]:
use_normalfindings = (data_liver['io_tokens'].apply(lambda x:x in normal_obs_finding_list)) & (data_liver['obs_certainty'] == 0)
use_abnormalfindings = (data_liver['io_tokens'].apply(lambda x:x in abnormal_obs_finding_list)) & (data_liver['obs_certainty'] != 0)
available_segmentation_list = glob.glob('../data/liver_seg_img/*.nii.gz') 
available_segmentation_list = [os.path.basename(p) for p in available_segmentation_list]
use_segmentation = data_liver['jmid_name'].apply(lambda x:x in available_segmentation_list)
# use_abnormalfindings = (data_liver['clin_findings'].str.contains('癌') | data_liver['clin_findings'].str.contains('IPMN'))
# use_certainty = ((data_liver['obs_certainty'] == 0) | (data_liver['obs_certainty'] == 4) )
data_liver['use_training'] = (use_normalfindings|use_abnormalfindings) & use_segmentation 
data_liver['label'] = 'not used'
data_liver.loc[use_abnormalfindings,'label'] = data_liver.loc[use_abnormalfindings,'io_tokens'].apply(lambda x:abnormal_obs_finding_list_renamed[abnormal_obs_finding_id[abnormal_obs_finding_list.index(x)]])
data_liver.loc[use_normalfindings,'label'] = 'nofinding'
tmp = data_liver.groupby('jmid_name')['label'].apply(lambda x: x.tolist())
data_liver.drop('label',axis=1,inplace=True)
data_liver = data_liver.merge(tmp,how='left',on='jmid_name')


In [ ]:
data_liver = to_multi_label(data_liver,abnormal_obs_finding_list_renamed)

data_liver = data_liver.drop_duplicates(subset='jmid_name')
print(data_liver.shape)
data_liver_use = data_liver[data_liver['use_training'] == True].reset_index(drop=True)
data_liver_use['file'] = data_liver_use['jmid_name']
data_liver_use['jmid_name'] = data_liver_use['jmid_name'].apply(lambda x:'_'.join(x.split('_')[:-1])+'_0000.nii.gz')

print(data_liver_use.shape)
##元の所見文と結合
data_liver_use = data_liver_use.merge(df,on='jmid_name',how='left').drop_duplicates(subset='jmid_name')
print(data_liver_use.shape)
data_liver_use[abnormal_obs_finding_list_renamed+['nofinding']].sum(axis=0)

In [ ]:
## cleanなラベルは検証データとして使用する。学習データはそれ以外を使用。
clean_df = pd.read_csv('../data/liver_dataset_test_clean.csv')
jmid_files_clean = clean_df['file'].values
tmp = data_liver_use[data_liver_use['jmid_name'].apply(lambda x:x in jmid_files_clean)]
test_groups = tmp['FACILITY_CODE'].astype(str)+tmp['ACCESSION_NUMBER'].astype(str)
groups = data_liver_use['FACILITY_CODE'].astype(str)+data_liver_use['ACCESSION_NUMBER'].astype(str)


df_train = data_liver_use[groups.apply(lambda x:x not in test_groups.tolist())]
df_test =  data_liver_use[groups.apply(lambda x:x in test_groups.tolist())]


columns = ['file','organ','all_tokens','io_tokens','ae_tokens','obs_certainty','FINDING','FINDING_JSON','FACILITY_CODE','ACCESSION_NUMBER']+abnormal_obs_finding_list_renamed+['nofinding']
df_train[columns].sort_values(by='file').reset_index(drop=True).to_csv('../data/liver_dataset_train_multi5.csv',index=False)
df_test[columns].sort_values(by='file').reset_index(drop=True).to_csv('../data/liver_dataset_test_multi5.csv',index=False)